## Sagnac Frequency - Backscatter Correction - Frequency Domain

## Imports

In [ ]:
import os
import gc
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, date
from pandas import DataFrame, read_pickle, date_range, concat, read_csv
from obspy import UTCDateTime, read
from scipy.signal import hilbert

In [ ]:
from functions.load_romy_raw_data import __load_romy_raw_data
from functions.hilbert_frequency_estimator import __hilbert_frequency_estimator
from functions.get_fft_values import __get_fft_values
from functions.get_time_intervals import __get_time_intervals
from functions.backscatter_correction import __backscatter_correction
from functions.sine_frequency_estimator import __sine_frequency_estimator

In [ ]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'

## Configurations

In [ ]:
config = {}

config['ring'] = "V"

config['seed'] = f"BW.DROMY..FJ{config['ring']}"

config['path_to_sds'] = archive_path+"romy_archive/"

# test U
config['tbeg'] = UTCDateTime("2023-09-19 00:00")
config['tend'] = UTCDateTime("2023-09-19 02:00")

# test V
config['tbeg'] = UTCDateTime("2024-05-22 16:00")
config['tend'] = UTCDateTime("2024-05-22 19:00")

# test Z
# config['tbeg'] = UTCDateTime("2024-08-01 15:30")
# config['tend'] = UTCDateTime("2024-08-01 19:30")


# define nominal sagnac frequency of rings
config['ring_sagnac'] = {"U":303.05, "V":447.5, "W":447.5, "Z":553.5}
config['nominal_sagnac'] = config['ring_sagnac'][config['ring']]

## path to Sagnac data
config['path_to_autodata'] = archive_path+f"romy_autodata/"

config['path_to_data'] = data_path+"sagnac_frequency/data/"

config['path_to_figs'] = data_path+"sagnac_frequency/figures/"

config['interval'] = 120 ## seconds

config['method'] = "fft" ## welch | multitaper | fft

config['ddt'] = 60

config['fband'] = 2

config['cm_value'] = 1.033

# set if amplitdes are corrected with envelope
config['correct_amplitudes'] = True

# set prewhitening factor (to avoid division by zero)
config['prewhitening'] = 0.1

# select frequency estimation mode
config['mode'] = "both" # "hilbert" | "sine" | "both"


### Load Archive Files

In [ ]:
sagn = __load_romy_raw_data(f"BW.DROMY..FJ{config['ring']}",
                            config['tbeg']-2*config['ddt'],
                            config['tend']+2*config['ddt'],
                            config['path_to_sds']
                           )
mon1 = __load_romy_raw_data("BW.DROMY..F1V",
                            config['tbeg']-2*config['ddt'],
                            config['tend']+2*config['ddt'],
                            config['path_to_sds']
                           )
mon2 = __load_romy_raw_data("BW.DROMY..F2V",
                            config['tbeg']-2*config['ddt'],
                            config['tend']+2*config['ddt'],
                            config['path_to_sds']
                           )

In [ ]:
# for ss in [sagn, mon1, mon2]:
#     ss.write(root_path+f"Downloads/mseed/DROMY_{ss[0].stats.channel}_{config['tbeg'].date}.mseed)

### Load Local Files

In [ ]:
# sagn = read(root_path+"Downloads/mseed/DROMY_FJU_2023-09-19.mseed")
# mon1 = read(root_path+"Downloads/mseed/DROMY_F1V_2023-09-19.mseed")
# mon2 = read(root_path+"Downloads/mseed/DROMY_F2V_2023-09-19.mseed")

# sagn = read(root_path+"Downloads/mseed/DROMY_FJU_2023-09-01.mseed")
# mon1 = read(root_path+"Downloads/mseed/DROMY_F1V_2023-09-01.mseed")
# mon2 = read(root_path+"Downloads/mseed/DROMY_F2V_2023-09-01.mseed")

# sagn.trim(config['tbeg'], config['tbeg']+1800)
# mon1.trim(config['tbeg'], config['tbeg']+1800)
# mon2.trim(config['tbeg'], config['tbeg']+1800)

# sagn = read(root_path+f"Downloads/mseed/DROMY_FJ{config['ring']}_{config['tbeg'].date}.mseed")
# mon1 = read(root_path+f"Downloads/mseed/DROMY_F1V_{config['tbeg'].date}.mseed")
# mon2 = read(root_path+f"Downloads/mseed/DROMY_F2V_{config['tbeg'].date}.mseed")

# for st0 in [sagn, mon1, mon2]:
#     for tr in st0:
#         tr.data = tr.data*0.59604645e-6 # V / count  [0.59604645ug  from obsidian]


In [ ]:
sagn.plot();
mon1.plot();
mon2.plot();

## Processing

In [ ]:
# def __hilbert_frequency_estimator2(st, nominal_sagnac, fband, cut=0):

#     from scipy.signal import hilbert
#     import numpy as np

#     st0 = st.copy()

#     ## extract sampling rate
#     df = st0[0].stats.sampling_rate

#     ## define frequency band around Sagnac Frequency
#     f_lower = nominal_sagnac - fband
#     f_upper = nominal_sagnac + fband

#     ## bandpass with butterworth around Sagnac Frequency
#     st0 = st0.detrend("linear")
#     st0 = st0.taper(0.01, type='cosine')
#     st0 = st0.filter("bandpass", freqmin=f_lower, freqmax=f_upper, corners=4, zerophase=True)

#     ## estimate instantaneous frequency with hilbert
#     signal = st0[0].data

#     ## calulcate hilbert transform
#     hil0 = hilbert(signal)

#     ## extract imaginary part of hilbert transform
#     hil = np.imag(hil0)

#     ## calculate derivatives
#     d_hil = np.gradient(hil, edge_order=1)
#     d_sig = np.gradient(signal, edge_order=1)

#     delta_f_full = (signal * d_hil - d_sig * hil) / (2*np.pi*np.sqrt(signal**2 + hil**2))

#     ## extract real part
#     instantaneous_frequency = np.real(delta_f_full)

#     del hil0, hil, d_hil, d_sig
#     ## cut first and last 5% (corrupted data)
#     # dd = int(cut*len(instantaneous_frequency))
#     dd = int(cut*df)
#     insta_f_cut = instantaneous_frequency[dd:-dd]

#     ## get times
#     t = st0[0].times()
#     t_mid = t[int((len(t))/2)]

#     ## averaging of frequencies
#     insta_f_cut_avg = np.mean(insta_f_cut)
#     # insta_f_cut_avg = np.median(insta_f_cut)

#     return t_mid, insta_f_cut_avg

In [ ]:
times = __get_time_intervals(config['tbeg'], config['tend'], interval_seconds=config['interval'], interval_overlap=0)

## Computing

In [ ]:
gc.collect();

In [ ]:
# overall samples
NN = len(times)

# prepare output arrays
fs, fh, ff = np.ones(NN)*np.nan, np.ones(NN)*np.nan, np.ones(NN)*np.nan
ac, dc, ph, st = np.ones(NN)*np.nan, np.ones(NN)*np.nan, np.ones(NN)*np.nan, np.ones(NN)*np.nan

ph_wrap = np.ones(NN)*np.nan

# prepare output dataframe
out_df = DataFrame()
out_df['time1'] = list(zip(*times))[0]
out_df['time2'] = list(zip(*times))[1]

for _k, _st in enumerate([sagn, mon1, mon2]):

    print(" -> processing ", _k, "...")

    for _n, (t1, t2) in enumerate(times):

        # _dat = _st.copy().trim(t1, t2)
        _dat = _st.copy().trim(t1-config['ddt'], t2+config['ddt'])

        # estimate AC and DC values in frequency domain
        fs[_n], ac[_n], dc[_n], ph[_n] = __get_fft_values(_dat[0].data,
                                                          _dat[0].stats.delta,
                                                          config['nominal_sagnac']
                                                         )

        # correct amplitudes with envelope
        if config['correct_amplitudes']:
            for tr in _dat:
                # scale by envelope
                env = abs(hilbert(tr.data)) + config['prewhitening']
                tr.data = tr.data / env


        if config['mode'] in ["hilbert", "both"]:
            # estimate instantaneous frequency average via hilbert
            t, fh[_n], _, st[_n] = __hilbert_frequency_estimator(_dat,
                                                                 config['nominal_sagnac'],
                                                                 fband=config['fband'],
                                                                 cut=config['ddt']
                                                                 )

        elif config['mode'] in ["sine", "both"]:
            # estimate instantaneous frequency average via sine fit
            t, fs[_n], _, _ = __sine_frequency_estimator(_dat,
                                                          config['nominal_sagnac'],
                                                          fband=config['fband'],
                                                          Tinterval=config['interval'],
                                                          Toverlap=0,
                                                          plot=False,
                                                         )
        if config['mode'] == "sine":
            ff[_n] = fs[_n]
        else:
            ff[_n] = fh[_n]

        # estimate DC and AC based on time series (time domain)
        # dc[_n] = np.median(_dat)
        # dc[_n] = np.mean(_dat)
        # ac[_n] = np.percentile(_dat[0].data, 99.9) - np.percentile(_dat[0].data, 100-99.9)

    # store wrapped phase
    ph_wrap = ph

    # store unwrapped phase
    ph = np.unwrap(ph)

    # fill output dataframe
    if _k == 0:
        out_df['fj_fs'], out_df['fj_ac'], out_df['fj_dc'], out_df['fj_ph'], out_df['fj_st'] = ff, ac, dc, ph, st
        out_df['fj_phw'] = ph_wrap
        if config['mode'] == "both":
            out_df['fj_sine'], out_df['fj_hilb'] = fs, fh
    elif _k == 1:
        out_df['f1_fs'], out_df['f1_ac'], out_df['f1_dc'], out_df['f1_ph'], out_df['f1_st'] = ff, ac, dc, ph, st
        out_df['f1_phw'] = ph_wrap
        if config['mode'] == "both":
            out_df['fj_sine'], out_df['fj_hilb'] = fs, fh
    elif _k == 2:
        out_df['f2_fs'], out_df['f2_ac'], out_df['f2_dc'], out_df['f2_ph'], out_df['f2_st'] = ff, ac, dc, ph, st
        out_df['f2_phw'] = ph_wrap
        if config['mode'] == "both":
            out_df['fj_sine'], out_df['fj_hilb'] = fs, fh

# prepare values for backscatter correction

# AC/DC ratios
m01 = out_df.f1_ac / out_df.f1_dc
m02 = out_df.f2_ac / out_df.f2_dc

# phase difference
phase0 = out_df.f1_ph - out_df.f2_ph
phase0 = np.unwrap(out_df.f1_phw) - np.unwrap(out_df.f2_phw)

# obseved Sagnac frequency with backscatter
w_obs = out_df.fj_fs

# apply backscatter correction
out_df['w_s'], out_df['bscorrection'], out_df['term'] = __backscatter_correction(m01, m02,
                                                                                 phase0,
                                                                                 w_obs,
                                                                                 config['nominal_sagnac'],
                                                                                 cm_filter_factor=config['cm_value'],
                                                                                 )


In [ ]:
def __makeplot(df):

    Nrow, Ncol = 5, 1

    font = 12
    ms = 15

    fig, ax = plt.subplots(Nrow, Ncol, figsize=(15, 10), sharex=True)

    t_axis = (np.array(range(len(df['fj_fs'])))+0.5)*config['interval']/60

    ax[0].scatter(t_axis, df['fj_fs'], zorder=2, s=ms, label="fj")
    ax[0].scatter(t_axis, df['f1_fs'], zorder=2, s=ms, label="f1")
    ax[0].scatter(t_axis, df['f2_fs'], zorder=2, s=ms, label="f2")
    ax[0].set_ylabel("$\delta$f (Hz)")
    ax[0].ticklabel_format(useOffset=False)

    # ax[1].scatter(t_axis, df['fj_ac'], zorder=2, s=ms, label="fj")
    ax[1].scatter(t_axis, df['f1_ac']*1e3, zorder=2, s=ms, label="f1", color="tab:orange")
    ax[1].scatter(t_axis, df['f2_ac']*1e3, zorder=2, s=ms, label="f2", color="tab:green")
    ax[1].set_ylabel("AC (mV)")
    ax[1].set_ylim(0, 4)

    ax11 = ax[1].twinx()
    ax11.scatter(t_axis, df['fj_ac'], zorder=2, s=ms, label="fj")
    ax11.set_ylabel("AC (V)")
    ax11.spines['right'].set_color('tab:blue')
    ax11.yaxis.label.set_color('tab:blue')
    ax11.tick_params(axis='y', colors='tab:blue')
    ax11.set_yticks(np.linspace(ax11.get_yticks()[0], ax11.get_yticks()[-1], len(ax[1].get_yticks())))

    # ax[2].scatter(t_axis, df['fj_dc'], zorder=2, s=ms, label="fj")
    ax[2].scatter(t_axis, df['f1_dc'], zorder=2, s=ms, label="f1", color="tab:orange")
    ax[2].scatter(t_axis, df['f2_dc'], zorder=2, s=ms, label="f2", color="tab:green")
    ax[2].set_ylabel("DC (V)")
    # ax[2].set_ylim(0.7, 1)

    ax21 = ax[2].twinx()
    ax21.scatter(t_axis, df['fj_dc'], zorder=2, s=ms, label="fj")
    ax21.set_ylabel("DC (V)")
    ax21.spines['right'].set_color('tab:blue')
    ax21.yaxis.label.set_color('tab:blue')
    ax21.tick_params(axis='y', colors='tab:blue')
    ax21.set_yticks(np.linspace(ax21.get_yticks()[0], ax21.get_yticks()[-1], len(ax[2].get_yticks())))

    # ax[3].scatter(t_axis, df['fj_ac']/df['fj_dc'], zorder=2, s=ms, label="fj")
    ax[3].scatter(t_axis, df['f1_ac']/df['f1_dc'], zorder=2, s=ms, label="f1", color="tab:orange")
    ax[3].scatter(t_axis, df['f2_ac']/df['f2_dc'], zorder=2, s=ms, label="f2", color="tab:green")
    ax[3].set_ylabel("AC/DC")
    ax[3].set_ylim(0, 5e-3)

    ax31 = ax[3].twinx()
    ax31.scatter(t_axis, df['fj_ac']/df['fj_dc'], zorder=2, s=ms, label="fj")
    ax31.set_ylabel("AC/DC")
    ax31.spines['right'].set_color('tab:blue')
    ax31.yaxis.label.set_color('tab:blue')
    ax31.tick_params(axis='y', colors='tab:blue')
    ax31.set_yticks(np.linspace(ax31.get_yticks()[0], ax31.get_yticks()[-1], len(ax[3].get_yticks())))

    ax[4].scatter(t_axis, df['f1_ph']-df['f2_ph'], color="k", zorder=2, s=ms, label="f1-f2")
    ax[4].set_ylabel("$\Delta$ Phase (rad)")
    # ax[4].set_yscale("log")

    ax[4].set_xlabel("Time (min)")

    for _n in range(Nrow):
        ax[_n].grid(ls=":", zorder=0)
        ax[_n].legend(loc=1, ncol=3)

    plt.show();
    return fig

fig = __makeplot(out_df);

# fig.savefig(config['path_to_figs']+f"SF_BS_{config['ring']}_{method}_values.png", format="png", dpi=150, bbox_inches='tight')


In [ ]:
def __makeplot2(df):

    Nrow, Ncol = 5, 1

    font = 12
    ms = 15

    fig, ax = plt.subplots(Nrow, Ncol, figsize=(15, 10), sharex=True)

    t_axis = (np.array(range(len(df['fj_fs'])))+0.5)*config['interval']/60

    ax[0].plot(t_axis, df['f1_fs'], zorder=2, label="f1", color="tab:orange")
    ax[0].plot(t_axis, df['f2_fs'], zorder=2, label="f2", color="tab:green")
    ax[0].set_ylabel("$\delta$f (Hz)")
    ax[0].ticklabel_format(useOffset=False)


    ax00 = ax[0].twinx()
    ax00.plot(t_axis, df['fj_fs'], zorder=2, label="fj", color="tab:blue")
    ax00.plot(t_axis, df['w_s'], zorder=2, label="bs", ls="--", color="tab:blue")
    ax00.set_ylabel("$\delta$f (Hz)")
    ax00.spines['right'].set_color('tab:blue')
    ax00.yaxis.label.set_color('tab:blue')
    ax00.tick_params(axis='y', colors='tab:blue')
    ax00.set_yticks(np.linspace(ax00.get_yticks()[0], ax00.get_yticks()[-1], len(ax[0].get_yticks())))
    ax00.ticklabel_format(useOffset=False)

    ax[1].plot(t_axis, df['f1_ac']*1e3, zorder=2, label="f1", color="tab:orange")
    ax[1].plot(t_axis, df['f2_ac']*1e3, zorder=2, label="f2", color="tab:green")
    ax[1].set_ylabel("AC (mV)")
    ax[1].set_ylim(0, 4)

    ax11 = ax[1].twinx()
    ax11.plot(t_axis, df['fj_ac'], zorder=2, label="fj")
    ax11.set_ylabel("AC (V)")
    ax11.spines['right'].set_color('tab:blue')
    ax11.yaxis.label.set_color('tab:blue')
    ax11.tick_params(axis='y', colors='tab:blue')
    ax11.set_yticks(np.linspace(ax11.get_yticks()[0], ax11.get_yticks()[-1], len(ax[1].get_yticks())))

    ax[2].plot(t_axis, df['f1_dc'], zorder=2, label="f1", color="tab:orange")
    ax[2].plot(t_axis, df['f2_dc'], zorder=2, label="f2", color="tab:green")
    ax[2].set_ylabel("DC (V)")

    ax21 = ax[2].twinx()
    ax21.plot(t_axis, df['fj_dc'], zorder=2, label="fj")
    ax21.set_ylabel("DC (V)")
    ax21.spines['right'].set_color('tab:blue')
    ax21.yaxis.label.set_color('tab:blue')
    ax21.tick_params(axis='y', colors='tab:blue')
    ax21.set_yticks(np.linspace(ax21.get_yticks()[0], ax21.get_yticks()[-1], len(ax[2].get_yticks())))

    ax[3].plot(t_axis, df['f1_ac']/df['f1_dc'], zorder=2, label="f1", color="tab:orange")
    ax[3].plot(t_axis, df['f2_ac']/df['f2_dc'], zorder=2, label="f2", color="tab:green")
    ax[3].set_ylabel("AC/DC")
    ax[3].set_ylim(0, 5e-3)

    ax31 = ax[3].twinx()
    ax31.plot(t_axis, df['fj_ac']/df['fj_dc'], zorder=2, label="fj")
    ax31.set_ylabel("AC/DC")
    ax31.spines['right'].set_color('tab:blue')
    ax31.yaxis.label.set_color('tab:blue')
    ax31.tick_params(axis='y', colors='tab:blue')
    ax31.set_yticks(np.linspace(ax31.get_yticks()[0], ax31.get_yticks()[-1], len(ax[3].get_yticks())))

    ax[4].plot(t_axis, df['f1_ph']-df['f2_ph'], color="k", zorder=2, label="f1-f2")
    ax[4].set_ylabel("$\Delta$ Phase (rad)")
    # ax[4].set_yscale("log")

    ax[4].set_xlabel("Time (min)")

    for _n in range(Nrow):
        ax[_n].grid(ls=":", zorder=0)
        ax[_n].legend(loc=1, ncol=3)

    plt.show();
    return fig

fig = __makeplot2(out_df);

# fig.savefig(config['path_to_figs']+f"SF_BS_{config['ring']}_{method}_values.png", format="png", dpi=150, bbox_inches='tight')


In [ ]:
plt.plot(out_df.fj_sine)
plt.plot(out_df.fj_hilb)


## Beat Data

In [ ]:
def __load_beat(tbeg, tend, ring, path_to_data):

    from datetime import date
    from pandas import read_pickle, concat, DataFrame, date_range
    from obspy import UTCDateTime

    tbeg, tend = UTCDateTime(tbeg), UTCDateTime(tend)

    dd1 = date.fromisoformat(str(tbeg.date))
    dd2 = date.fromisoformat(str(tend.date))

    year = str(tbeg.year)

    df = DataFrame()
    for dat in date_range(dd1, dd2):
        file = f"{year}/R{ring}/FJ{ring}_"+str(dat)[:10].replace("-", "")+".pkl"
        try:
            df0 = read_pickle(path_to_data+file)
            df = concat([df, df0])
        except:
            print(f"error for {file}")

    ## trim to defined times
    df = df[(df.times_utc >= tbeg) & (df.times_utc < tend)]

    ## correct seconds
    df['times_utc_sec'] = [abs(tbeg - UTCDateTime(_t))  for _t in df['times_utc']]

    return df

In [ ]:
# t1, t2 = UTCDateTime(out_df['time1'][0]), UTCDateTime(out_df['time1'][0])+len(out_df['time1'])*interval

# # data = __load_beat(t1, t2, config['seed'].split(".")[3][2], config['path_to_data'])
# data = out_df

# w_obs = data['fj_fs']